In [1]:
import numpy as np
import math

In [2]:
class CoxRossRubinstein:
    """
        Klasa będąca realizacją modelu CRR (Coxa-Rossa-Rubinsteina)
        służąca do wyceny bardziej skomplikowanych wypłat.
        
        Parametry:
            * S0 - cena początkowa
            * K - cena wykonania (strike)
            * T - horyzont czasowy
            * rate - stopa procentowa
            * u, d - współczynniki wzrostu/spadku ceny
            * only_lastLayer - flaga wskazująca czy cena wyliczana jest w oparciu tylko o ostatnią warstwę,
                               czy wykorzystywana będzie większa część trajektorii.
    """
    def __init__(self, S0, K, T, r, M, only_lastLayer=True) -> np.array:
        
        self.S0 = S0  
        self.strike_price = K    
        self.T = T    
        self.rate = r 
        
        self.only_lastLayer = only_lastLayer 
        
        if M is None: self.n_layers = T+1 # zakładamy, że transakcje mogą odbywać się w momentach {0,1,2,...,T}
        else: self.n_layers = M # ale dopuszczam rozszerzenie implementacji o inne możliwości

        self.p = (1+r-d)/(u-d)
        
    def payoff(self, stock_price, opt_type) -> np.array:
        """
        Funkcja wyceniająca wartość wypłaty
        z opcji typu call lub put.
        """
        if opt_type == "call": coef = 1 # wycena opcji typu call
        else: coef = -1                 # wycena opcji typu put
        return np.maximum(coef*(stock_price-self.strike_price), 0)
        
    def initialize_layers(self) -> np.array:     
        """
       Funkcja obliczająca wartości cen 
       na ostatnim poziomie drzewa 
       rekombinującego o wysokości n_layers
        """
        if self.only_lastLayer:
            # liści drzewa jest o 1, więcej niż poziomów
            # (bo każdy liść ma u w potędze wart. 
            # ze zbioru {0,1,...,height})
            s_grid = [None] * self.n_layers
            # Inicjalizacja pierwszego elementu 
            # ostatniej warstwy drzewa
            s_grid[0] = self.S0*(self.up_factor**self.T)
            # Obliczenie kolejnych elementów poprzez 
            # przemnożenie pierwszego przez iloraz d/u.
            for i in range(1, self.n_layers):
                s_grid[i] = s_grid[i-1] * (self.down_factor/self.up_factor)
        else:
            # inicjalizacja wszystkich warstw zabezpieczająca możliwość
            # rozszerzenia implementacji o przypadek, w którym wypłata
            # miałaby zależeć od większej części lub całej trajektorii cen akcji.
            s_grid = [np.array([self.S0*(self.down_factor**i)*(self.up_factor**(n-i))\
                                for i in range(n+1)]) for n in range(self.n_layers)]
        return s_grid
    
    
    def calculate_priceEU(self, opt_type) -> float:
        """
        Funkcja zwracająca wartości cen
        obliczone zadaną metodą.
        """
        if self.only_lastLayer:
            tree = [[None]*(self.n_layers-i) for i in reversed(range(1, self.n_layers))]
            last_layer = np.array(self.initialize_layers())
            opt_price = self.payoff(last_layer, opt_type)
            tree.append(opt_price)

            ## wzór rekurencyjny
            for layer_idx in reversed(range(self.n_layers-1)):
                for item_idx in range(len(tree[layer_idx])):
                    db = (1+self.rate)**(layer_idx)/(1+self.rate)**(layer_idx+1)
                    tree[layer_idx][item_idx] = db*(self.p*tree[layer_idx+1][item_idx]+\
                                                      (1-self.p)*tree[layer_idx+1][item_idx+1])
        else: 
            return None # Aby możliwa była obsługa przypadku, kiedy cena jest funkcją więcej 
                        # niż tylko ostatniej warstwy funkcja ta musi być znana.
        return tree[0][0]

    
    def theoretical_price(self, opt_type):
        """
        Funkcja obliczająca teoretyczne
        wartości cen opcji.
        """
        value = 0
        last_layer = np.array(self.initialize_layers())
        opt_price = self.payoff(last_layer, opt_type)
        opt_price = opt_price[::-1]
        
        for j in range(len(opt_price)):
            value += (math.factorial(self.T)*self.p**j*(1-self.p)**(self.T-j)*opt_price[j])\
            /(math.factorial(j)*math.factorial(self.T-j))
        return value/(1+self.rate)**self.T

In [3]:
# Przykład z wykładów UW (pozwalający zweryfikować poprawność działania implementacji)
crr = CoxRossRubinstein(S0=100, K=90, T=2, u=1.3, d=0.8, r=0.1)

print("Cena opcji kupna wyznaczona metodą CRR: ", crr.calculate_priceEU("call"),\
      "\nCena opcji sprzedaży wyznaczona metodą CRR: ", crr.calculate_priceEU("put"),\
      "\nTeoretyczna cena kupna: ", crr.calculate_priceEU("call"),\
      "\nTeoretyczna cena sprzedaży: ", crr.calculate_priceEU("put"))

Cena opcji kupna wyznaczona metodą CRR:  29.05785123966945 
Cena opcji sprzedaży wyznaczona metodą CRR:  3.438016528925616 
Teoretyczna cena kupna:  29.05785123966945 
Teoretyczna cena sprzedaży:  3.438016528925616


In [4]:
crr = CoxRossRubinstein(S0=100, K=90, T=10, u=1.3, d=0.8, r=0.1)

print("Cena opcji kupna wyznaczona metodą CRR: ", crr.calculate_priceEU("call"),\
      "\nCena opcji sprzedaży wyznaczona metodą CRR: ", crr.calculate_priceEU("put"),\
      "\nTeoretyczna cena kupna: ", crr.calculate_priceEU("call"),\
      "\nTeoretyczna cena sprzedaży: ", crr.calculate_priceEU("put"))

Cena opcji kupna wyznaczona metodą CRR:  66.97006486520782 
Cena opcji sprzedaży wyznaczona metodą CRR:  1.6689609138656176 
Teoretyczna cena kupna:  66.97006486520782 
Teoretyczna cena sprzedaży:  1.6689609138656176
